In [8]:
from pathlib import Path
import os
import sqlite3

import s3fs
import pandas as pd

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)

## Create and Load Measurements Table

In [9]:
def create_measurements_table(conn):
    sql = """
    CREATE TABLE IF NOT EXISTS measurements (
        visit_id integer NOT NULL,
        person_id text NOT NULL,
        quantity text,
        reading real,
        FOREIGN KEY (visit_id) REFERENCES visits (visit_id),
        FOREIGN KEY (person_id) REFERENCES people (people_id)
        );
    """

    c = conn.cursor()
    c.execute(sql)
    
def load_measurements_table(conn):
    create_measurements_table(conn)
    df = pd.read_csv('data/external/tidynomicon/measurements.csv')
    measurements = df.values
    c = conn.cursor()
    c.execute('DELETE FROM measurements;') # Delete data if exists
    c.executemany('INSERT INTO measurements VALUES (?,?,?,?)', measurements)

## Create and Load People Table

In [10]:
def create_people_table(conn):
    sql = """
    CREATE TABLE IF NOT EXISTS people (
        person_id text NOT NULL,
        personal_name text,
        family_name text
        );
    """
    ## TODO: Complete SQL
    c = conn.cursor()
    c.execute(sql)
    
def load_people_table(conn):
    create_people_table(conn)
    df = pd.read_csv('data/external/tidynomicon/person.csv')
    people = df.values
    c = conn.cursor()
    c.execute('DELETE FROM people;') # Delete data if exists
    c.executemany('INSERT INTO people VALUES (?,?,?)', people)

## Create and Load Sites Table

In [11]:
def create_sites_table(conn):
    sql = """
    CREATE TABLE IF NOT EXISTS sites (
        site_id text PRIMARY KEY,
        latitude double NOT NULL,
        longitude double NOT NULL
        );
    """

    c = conn.cursor()
    c.execute(sql)

def load_sites_table(conn):
    create_sites_table(conn)
    df = pd.read_csv('data/external/tidynomicon/site.csv')
    sites = df.values
    c = conn.cursor()
    c.execute('DELETE FROM sites;') # Delete data if exists
    c.executemany('INSERT INTO sites VALUES (?,?,?)', sites)

## Create and Load Visits Table

In [12]:
def create_visits_table(conn):
    sql = """
    CREATE TABLE IF NOT EXISTS visits (
        visit_id integer PRIMARY KEY,
        site_id text NOT NULL,
        visit_date text,
        FOREIGN KEY (site_id) REFERENCES sites (site_id)
        );
    """

    c = conn.cursor()
    c.execute(sql)

def load_visits_table(conn):
    create_visits_table(conn)
    df = pd.read_csv('data/external/tidynomicon/visited.csv')
    visits = df.values
    c = conn.cursor()
    c.execute('DELETE FROM visits;') # Delete data if exists
    c.executemany('INSERT INTO visits VALUES (?,?,?)', visits)

## Create DB and Load Tables

In [13]:
db_path = results_dir.joinpath('patient-info.db')
conn = sqlite3.connect(str(db_path))

create_measurements_table(conn)
create_people_table(conn)
create_sites_table(conn)
create_visits_table(conn)

load_people_table(conn) 
load_sites_table(conn)
load_visits_table(conn)
load_measurements_table(conn)

conn.commit()

In [14]:
query = """
SELECT 
    measurements.visit_id,
    measurements.person_id,
    people.personal_name,
    people.family_name,
    visits.visit_date,
    visits.site_id,
    measurements.quantity,
    measurements.reading,
    sites.latitude,
    sites.longitude
    
FROM measurements
JOIN 
    people ON people.person_id = measurements.person_id
JOIN 
    visits ON measurements.visit_id = visits.visit_id
JOIN 
    sites ON sites.site_id = visits.site_id

"""

pd.read_sql(query, conn)

,visit_id,person_id,personal_name,family_name,visit_date,site_id,quantity,reading,latitude,longitude
0,619,dyer,William,Dyer,1927-02-08,DR-1,rad,9.82,-49.85,-128.57
1,619,dyer,William,Dyer,1927-02-08,DR-1,sal,0.13,-49.85,-128.57
2,622,dyer,William,Dyer,1927-02-10,DR-1,rad,7.80,-49.85,-128.57
3,622,dyer,William,Dyer,1927-02-10,DR-1,sal,0.09,-49.85,-128.57
4,734,pb,Frank,Pabodie,1930-01-07,DR-3,rad,8.41,-47.15,-126.72
5,734,lake,Anderson,Lake,1930-01-07,DR-3,sal,0.05,-47.15,-126.72
6,734,pb,Frank,Pabodie,1930-01-07,DR-3,temp,-21.50,-47.15,-126.72
7,735,pb,Frank,Pabodie,1930-01-12,DR-3,rad,7.22,-47.15,-126.72
8,735,pb,Frank,Pabodie,1930-01-12,DR-3,sal,0.06,-47.15,-126.72
9,735,pb,Frank,Pabodie,1930-01-12,DR-3,temp,-26.00,-47.15,-126.72


In [15]:
conn.close()